# Day 1 - API Access

This is just an introductory notebook to make sure you have API access. We simply test each API and ensure we're getting a response. If you run into any issues here, please reach out to us!

Please make sure that you have received your environment files and you've placed them at the root of this repository. That is, a `.env` and a `google-api.json` file should be in the root of this repository alongside the `setup.py` and `pyproject.toml` files already present in the repository. You do not have to modify these files unless explicitly instructed to.

In [26]:
# Load environment variables
import dotenv
dotenv.load_dotenv("../../.env", override=True)

True

### Chat Completion APIs vs Text Completion APIs

Both OpenAI and Google have two different APIs we can use to make calls to their respective LLMs. Chat Completion APIs are conversational in nature, meaning the user and the model take turns to send messages (like chatGPT). Text Completion APIs on the other hand are meant to be singular prompts where we give a prompt and the model attempts to "complete" the text sequence.

Chat Completion APIs: [ChatCompletion (OpenAI)](https://platform.openai.com/docs/api-reference/chat), [ChatModel (Google)](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#chat_model_parameters)

Text Completion APIs: [Completion (OpenAI)](https://platform.openai.com/docs/api-reference/completions), [TextGeneration (Google)](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#text_model_parameters)

Note that the latest OpenAI models such as GPT-4 and GPT-3.5-turbo are only available through their chat completion API. Thus we exclusively use only the chat completion API for OpenAI models. In contrast, Google's PaLM 2 model is available for both APIs. Hence we primarily use the text completion API as [it supports several tasks](https://developers.generativeai.google/models/language). We use the chat completion API only for the chatbot example.

Our final model, an open-source model that we have hosted (Falcon) uses only a text generation API as recommended by [HuggingFace](https://huggingface.co/blog/falcon).

## OpenAI

### Completion

In [7]:
import openai

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "How do I make a margherita pizza?"},
]

chatbot_response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages 
)

print(messages[-1]["content"])
print(chatbot_response.choices[0].message["content"])

How do I make a margherita pizza?
To make a Margherita pizza, follow these steps:

Ingredients:

1) Pizza Dough:
- 2 1/4 teaspoons (or 1 packet) active dry yeast
- 1 1/3 cups warm water (around 110°F/45°C)
- 3 1/2 cups all-purpose flour
- 2 tablespoons olive oil
- 2 teaspoons sugar
- 1 teaspoon salt

2) Toppings:
- 1 cup tomato sauce (preferably San Marzano or homemade)
- 8 oz. fresh mozzarella cheese (sliced or torn into small pieces)
- 1/2 cup fresh basil leaves
- 2 tablespoons olive oil
- Salt, to taste

Instructions:

1. Prepare the pizza dough:
  a. In a small bowl, combine the warm water and sugar. Sprinkle the active dry yeast on top and let it sit for about 5 minutes, until it becomes foamy.
  b. In a large mixing bowl, combine the flour and salt. Make a well in the middle and pour in the yeast mixture and olive oil. Mix the ingredients together until a rough dough forms.
  c. Knead the dough on a floured surface for about 10 minutes, or until it becomes smooth and elastic. Alt

### ChatCompletion

The ChatCompletion API offers a ChatGPT-style messaging interface.

In [4]:
import openai
QUESTION = "How do I make a margherita pizza?"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": QUESTION},
    ],
  max_tokens=256,
  temperature=0.3
)

print(QUESTION)
print("---" * 30)
print(response["choices"][0]["message"]["content"])

How do I make a margherita pizza?
------------------------------------------------------------------------------------------
Here's a simple recipe for making a margherita pizza:

Ingredients:
- 1 pizza dough ball
- 1/2 cup tomato sauce
- 8 oz. fresh mozzarella cheese, sliced
- 1/4 cup fresh basil leaves
- 1 tbsp. olive oil
- Salt and pepper to taste

Instructions:
1. Preheat your oven to 450°F (230°C).
2. Roll out the pizza dough on a floured surface until it's about 1/4 inch thick.
3. Place the dough on a pizza pan or baking sheet.
4. Spread the tomato sauce evenly over the dough, leaving a small border around the edges.
5. Arrange the mozzarella slices on top of the sauce.
6. Drizzle the olive oil over the pizza.
7. Season with salt and pepper to taste.
8. Bake the pizza for 10-12 minutes, or until the crust is golden brown and the cheese is melted.
9. Remove the pizza from the oven and sprinkle the fresh basil leaves over the top.
10. Slice and serve hot.

Enjoy your homemade margh

## Google

In [13]:
from vertexai.preview.language_models import TextGenerationModel

QUESTION = "How do I make a margherita pizza?"

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(
    QUESTION,
    temperature=0.3,
    max_output_tokens=256,
)

print(QUESTION)
print("---" * 30)
print(response.text)

How do I make a margherita pizza?
------------------------------------------------------------------------------------------
**Ingredients:**

* 1 batch of pizza dough (homemade or store-bought)
* 2 cups of pizza sauce
* 1 cup of shredded mozzarella cheese
* 2 to 3 fresh tomatoes, sliced
* 1/2 cup of fresh basil leaves, torn
* 1/4 cup of olive oil
* Salt and pepper to taste

**Instructions:**

1. Preheat the oven to 450 degrees Fahrenheit.
2. If using homemade pizza dough, prepare the dough according to the recipe. If using store-bought pizza dough, let it come to room temperature for about 30 minutes before using.
3. Roll out the pizza dough into a 12-inch circle.
4. Spread the pizza sauce evenly over the dough.
5. Top with the mozzarella cheese, sliced tomatoes, and basil leaves.
6. Drizzle with olive oil and season with salt and pepper to taste.
7. Bake the pizza for 15 to 20 minutes, or until the crust is golden brown and the cheese is melted and bubbly.
8. Let the pizza cool for a

## Open Source LLM: Falcon-7B-Instruct

Note that we have set up hosting for this model separately, with the endpoint and api tokens being pre-set in the .env file for you. For a free, public endpoint, you could use: https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct

In [38]:
import requests
import os

headers = {"Authorization": f"Bearer {os.environ.get('HUGGINGFACEHUB_API_TOKEN')}"}

def query(payload):
    response = requests.request("POST", os.environ.get("HUGGINGFACEHUB_ENDPOINT"), headers=headers, json=payload)
    return response.json()

data = query({"inputs": "How do I make a margherita pizza?", "parameters": {"max_new_tokens": 250}})

print(data[0]['generated_text'])

How do I make a margherita pizza?
To make a margherita pizza, you will need pizza dough, tomato sauce, fresh mozzarella cheese, fresh basil leaves, and olive oil. Preheat your oven to 475°F. Roll out your pizza dough and spread tomato sauce on top. Add fresh mozzarella cheese and fresh basil leaves. Drizzle with olive oil and bake for 12-15 minutes. Enjoy!
